In [6]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from datetime import datetime

In [28]:
re_df = pd.DataFrame().assign(Raw_Price=[], Raw_Beds=[], Raw_Baths=[], 
                     Raw_Square_Feet=[], Raw_Year_Built=[], Raw_Garages=[], 
                     Raw_Lot_SqFt=[], Raw_Date_Sold=[], Raw_Days_On_Market=[])
months_dict = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 
               'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

In [30]:
i = 1
while i:
    url = f'https://www.mlslistings.com/Search/Result/6c8ec8db-d99b-4466-9bb2-f40f3593e7de/{i}'
    page = requests.get(url)
    driver = webdriver.Chrome()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    try:
        soup.find('strong', {'class': "listing-price d-block pull-left pr-25"}).text
    except:
        break
    price = [x.text for x in soup.find_all('strong', {'class': "listing-price d-block pull-left pr-25"})]
    baths = [x.text for x in soup.find_all('span', {'class': "listing-info-item font-size-sm line-height-base d-block pull-left pr-50 listing-baths"})]
    beds = [x.text for x in soup.find_all('span', {'class': "listing-info-item font-size-sm line-height-base d-block pull-left pr-50 listing-beds"})]
    sqft_yb = [x.text for x in soup.find_all('span', {'class': "listing-info-item font-size-sm line-height-base d-block pull-left pr-50 listing-sqft last"})]
    sqft = [sqft_yb[i] for i in range(len(sqft_yb)) if i % 2 == 0]
    yb = [sqft_yb[i] for i in range(len(sqft_yb)) if i % 2 == 1]
    garages = [x.text for x in soup.find_all('span', {'class': "listing-info-item font-size-sm line-height-base d-block pull-left pr-50 listing-garage"})]
    lot = [x.text for x in soup.find_all('span', {'class': "listing-info-item font-size-sm line-height-base d-block pull-left pr-50 listing-lot-size"})]
    date_sold = [x.text for x in soup.find_all('span', {'class': "listing-statusd-block pull-left pl-50 pr-1 status-marker status-closed"})]
    days_active = [x.text for x in soup.find_all('span', {'class': "listing-dom-block pull-left pl-25 hidden"})]
    new_df = pd.DataFrame().assign(Raw_Price=price, Raw_Beds=beds, Raw_Baths=baths, 
                 Raw_Square_Feet=sqft, Raw_Year_Built=yb, Raw_Garages=garages, 
                 Raw_Lot_SqFt=lot, Raw_Date_Sold=date_sold, Raw_Days_On_Market=days_active)
    re_df = pd.concat([re_df, new_df])
    i += 1 

In [32]:
try:
    re_df['Price'] = [int(x.replace('$','').replace(',','')) for x in re_df['Raw_Price']]
    re_df['Baths'] = [float(x.replace('\n','').replace('/','.').replace('Ba', '')) for x in re_df['Raw_Baths']]
    re_df['Beds'] = [int(x.replace('\n','').replace('Bd','')) for x in re_df['Raw_Beds']]
    re_df['Square_Feet'] = [int(x.replace('\n','').replace('Sq Ft','').replace(',','')) for x in 
                            re_df['Raw_Square_Feet']]
    re_df['Year_Built'] = [int(x.replace('\n', '').replace('Year Built', '')) for x in re_df['Raw_Year_Built']]
    re_df['Garages'] = [int(x.replace('\n', '').replace('Garage', '')) for x in re_df['Raw_Garages']]
    re_df['Lot_SqFt'] = [int(x.replace('\n','').replace('Sq Ft Lot','').replace(',','').replace('–','0')) if 
                         'Sq Ft' in x else int(float(x.replace('\n','').replace('Acres Lot','').replace(',','')
                                                     .replace('–','0')) * 45360) for x in re_df['Raw_Lot_SqFt']]
    re_df['Date_Sold'] = [f'{months_dict[x[0]]}/{x[1]}/{x[2]}' if len(x[1]) == 2 else 
                          f'{months_dict[x[0]]}/0{x[1]}/{x[2]}'for x in [y.replace('Sold on ','').replace(',','')
                                                                         .split() for y in re_df['Raw_Date_Sold']]]
    re_df['Days_On_Market'] = [int(x.strip(' Days on Site')) for x in re_df['Raw_Days_On_Market']]
    re_df = re_df.drop(columns=['Raw_Price','Raw_Beds','Raw_Baths','Raw_Square_Feet','Raw_Year_Built',
                                'Raw_Garages','Raw_Lot_SqFt','Raw_Date_Sold','Raw_Days_On_Market'])
except:
    pass

In [40]:
re_df.to_csv(f'mlssales{datetime.now().strftime("%y-%m-%d")}.csv')

In [39]:
re_df

,Price,Baths,Beds,Square_Feet,Year_Built,Garages,Lot_SqFt,Date_Sold,Days_On_Market
0,1000000,3.0,4,2889,2003,3,8786,07/14/2023,39
1,640000,2.1,2,1867,1990,2,4792,07/14/2023,30
2,999000,3.0,4,3103,2018,3,7339,07/12/2023,86
3,862000,2.1,4,2680,1999,3,8133,07/11/2023,44
4,630000,2.1,3,1888,2014,2,4147,07/11/2023,73
...,...,...,...,...,...,...,...,...,...
9,685000,2.0,2,1803,2004,2,7971,04/18/2023,114
10,740000,3.0,3,2504,2005,2,8028,04/18/2023,184
11,1200000,3.1,3,2962,2005,3,9265,04/17/2023,136
12,625000,2.0,3,1327,2000,2,6199,04/17/2023,116
